In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import embedders

In [59]:
import embedders.coordinate_learning


pm = embedders.manifolds.ProductManifold(signature=[(-1, 4)])
dists, adj = embedders.dataloaders.load_karate_club()
dists = dists / dists.max()
X_embed, losses = embedders.coordinate_learning.train_coords(pm, dists, burn_in_iterations=1000, training_iterations=9000)

Top CC has 34 nodes; original graph has 34 nodes.


  0%|          | 0/10000 [00:00<?, ?it/s]

In [91]:
# Make link prediction dataset
import networkx as nx
import torch

emb = []
for i in range(len(pm.x_embed)):
    for j in range(len(pm.x_embed)):
        joint_embed = torch.cat([X_embed[i], X_embed[j]])
        emb.append(joint_embed)

X = torch.stack(emb)
X = torch.cat([X, dists.flatten().unsqueeze(1)], dim=1)
print(X.shape)

y = torch.tensor(adj.flatten())

torch.Size([1156, 11])


In [109]:
# Try link prediction
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

MAX_DEPTH = 3
USE_SPECIAL_DIMS = True
# SIGNATURE = [(-1, 4), (-1, 4)]
SIGNATURE = [(-1, 4), (-1, 4), (0, 1)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pdt = embedders.tree_new.ProductSpaceDT(
    pm=embedders.manifolds.ProductManifold(SIGNATURE), 
    max_depth=MAX_DEPTH, 
    task="classification", 
    use_special_dims=USE_SPECIAL_DIMS
)
pdt.fit(X_train, y_train)
print("PDT", pdt.score(X_test, y_test).float().mean().item())

X_train, X_test, y_train, y_test = X_train.detach().numpy(), X_test.detach().numpy(), y_train.detach().numpy(), y_test.detach().numpy()

dt = DecisionTreeClassifier(max_depth=MAX_DEPTH)
dt.fit(X_train, y_train)
print("DT", dt.score(X_test, y_test))

PDT 1.0
DT 1.0


In [137]:
import embedders.coordinate_learning


pm = embedders.manifolds.ProductManifold(signature=[(-1, 2)])
dists, adj = embedders.dataloaders.load_adjnoun()
dists = dists / dists.max()
X_embed, losses = embedders.coordinate_learning.train_coords(pm, dists, burn_in_iterations=1000, training_iterations=9000)

Top CC has 112 nodes; original graph has 112 nodes.


  0%|          | 0/10000 [00:00<?, ?it/s]

In [138]:
# Make link prediction dataset

emb = []
for i in range(len(pm.x_embed)):
    for j in range(len(pm.x_embed)):
        joint_embed = torch.cat([X_embed[i], X_embed[j]])
        emb.append(joint_embed)

X = torch.stack(emb)
X = torch.cat([X, dists.flatten().unsqueeze(1)], dim=1)
print(X.shape)

y = torch.tensor(adj.flatten())

torch.Size([12544, 7])


In [139]:
# Try link prediction
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

MAX_DEPTH = 3
USE_SPECIAL_DIMS = True
# SIGNATURE = [(-1, 2), (-1, 2)]
SIGNATURE = [(-1, 2), (-1, 2), (0, 1)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pdt = embedders.tree_new.ProductSpaceDT(
    pm=embedders.manifolds.ProductManifold(SIGNATURE), 
    max_depth=MAX_DEPTH, 
    task="classification", 
    use_special_dims=USE_SPECIAL_DIMS
)
pdt.fit(X_train, y_train)
print("PDT", pdt.score(X_test, y_test).float().mean().item())

X_train, X_test, y_train, y_test = X_train.detach().numpy(), X_test.detach().numpy(), y_train.detach().numpy(), y_test.detach().numpy()

dt = DecisionTreeClassifier(max_depth=MAX_DEPTH)
dt.fit(X_train, y_train)
print("DT", dt.score(X_test, y_test))

PDT 1.0
DT 1.0
